**handson用資料としての注意点**

普通、同じセル上で何度も試行錯誤するので、最終的に上手くいったセルしか残らず、失敗したセルは残りませんし、わざわざ残しません。

今回はhandson用に 試行・思考過程を残したいと思い、エラーやミスが出ても下のセルに進んで処理を実行するようにしています。

notebookのセル単位の実行ができるからこそのやり方かもしれません。良い。

（下のセルから文は常体で書きます。）

kunai (@jdgthjdg)

---


# datetime型 をindexにして時系列表示する

（グラフにした時に、x軸が日付 2008/01/03 みたいになるようにする）

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#設定でDataFrameなどが長く表示されないようにします（画面領域の消費を抑えてhandsonをしやすくするため）
# 長い場合の途中の省略表示(...)を出す閾値の設定（折り返しとは無関係）
pd.set_option('max_rows',10)
pd.set_option('max_columns',20) # これを超えたら全部は表示しない。 A B C ... X Y Z のように途中を省く。

In [ ]:
df = pd.read_excel("../../../data/2008_kansai/ë╘ò▓âfü[â^2008(è╓É╝).xls", skiprows=1).iloc[:,:-2]
df.head(3)

現状のグラフは下。  

dfやseriesに .plot() で簡易な折れ線グラフができる。 

y軸のマイナスの値はあとで修正する  
x軸(index)は 0から振られた番号でしかない。  
→ 何年のデータなのかが分からない。  

In [ ]:
# 列(1つなので Series)の選択
df["彦根"]  # DataFrameではない。 Series(列*1 + index*1)

In [ ]:
df[  "彦根"  ].plot(figsize=(20,4)); # 行末の ; は出力の抑制をしているだけ。消して実行すると余計なものが見える

In [ ]:
df[   ["彦根"]   ]  # 最初はハマるが、リストで選択すると、複数列と判断されるので、 1列なのにSeriesではなく、 DataFrameになる
# 大げさな空白は分かりやすさのため

In [ ]:
df[   ["彦根"]   ].plot(figsize=(20,4));  # 行末の ; は出力の抑制をしているだけ。消して実行すると余計なものが見える

# x軸を日付にしたグラフが見たい！


データを見ると、 **datetimeに必要な日付要素が、バラバラの列に格納されている（レアなデータ）**  
    
for文で横一行づつ取得して、　年・月・日・時をまとめる？  
→ これは**行単位**での演算で、pandasの得意とする処理ではない。  
**かなり遅くなる**ので最終手段にする。
    

#### pandasの得意とする処理：列単位での演算（ df[ "大津" ] + df[ "京都" ] など ）

<br>

---


<br>

### 日付データを得るために2つのパターンを出す。後述するが、 pd.to_datetime という関数を使う。

1. 最初から各列をstringに変換してから、連結して日付をつくる
2. できるだけstringを使わずに連結し、最後だけstringに変換して日付をつくる

# パターン1：最初から各列をstringに変換してから、連結して日付をつくる

いつものpythonの文字列連結のノリでやると・・・

In [ ]:
"A" + "B" + "C"*3

In [ ]:
df["年"]+"/"+df["月"]+"/"+df["日"]+"/"+df["時"]  # Error

## **pd.read_XXX() したら、まず最初にデータの型(dtype)** を確認する

#### pandas で読み込んだら、まずは各列の型を見ること！
#### 希望的観測で、きっとこの型だろう！と思い込んでいると、あとあと困る・・・

In [ ]:
df["年"].dtype, df["月"].dtype,df["日"].dtype,df["時"].dtype,

In [ ]:
df.dtypes # めんどいならこっち

In [ ]:
df.iloc[:,:5].dtypes # ilocを利用する

int (数値)として読み込まれていたので、string に**キャスト（型の変換）** してからやってみる  
キャストは [astype](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html)


In [ ]:
df["年"].head(1)  # int64

In [ ]:
df["年"].astype(str).head(1)  # object

<br>

**strとintの見た目が同じなので分かりにくい**  
( dtype: object になっている )

floatなら小数点が付くので分かりやすい

In [ ]:
df["年"].astype(float).head(1)  # float64  .0 が付いている

### 文字列に変換したあとなら、あとはpandasが連結してくれる。 pandas 偉い。

In [ ]:
str_concat = df["年"].astype(str)+"/"+df["月"].astype(str)+"/"+df["日"].astype(str)+"/"+df["時"].astype(str)
str_concat.tail(3)

<br>
なんとかそれっぽくなった

# datetime型として読み込む [pd.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

とりあえず突っ込む

In [ ]:
pd.to_datetime(str_concat)  # Error

"年月日時をスラッシュ区切り" という特殊なフォーマットなので、読み取り方を教えてあげる  

**format** オプション 
https://docs.python.org/ja/3.7/library/datetime.html#strftime-and-strptime-behavior

**%m(月) と %M(分) に注意**  

よく使うものは以下  
%y	西暦下2桁。	00, 01, ..., 99	 
%Y	西暦4桁 	 2013, 2014, ..., 9998, 9999	(2)
%m	0埋めした月。	01, 02, ..., 12	 

%d	0埋めした10進数で表記した月中の日にち。	01, 02, ..., 31	 
%H	0埋めした10進数で表記した時 (24時間表記)。	00, 01, ..., 23	 
%M	0埋めした10進数で表記した分。	00, 01, ..., 59	 
%S	0埋めした10進数で表記した秒。	00, 01, ..., 59	(4)
%f	10進数で表記したマイクロ秒 (左側から0埋めされます)。	000000, 000001, ..., 999999	(5)



In [ ]:
pd.to_datetime(str_concat, format="%Y/%m/%d/%H")  # Error

<br>
<br>
<br>

### **ValueError: unconverted data remains: 4** とでたので調べると24時が怪しい  

<br>
<br>
<br>
そういえば、、、
<br>

%H のとり得る範囲 :  **00**, 01, ..., **23**

In [ ]:
df["時_0_23"] = df["時"]-1

### df\["時"\] に上書きしない理由
df\[ "時" \] = df\[ "時" \] - 1 のように、"時" 列に上書きもできるが、ミスで複数回実行してしまった場合に、結果が変わり原因の特定に苦労しかねない。(今回のケースなら-1になってエラーが出るから気付けるが。)  

fillna() など、複数回実行しても結果が変わらない（=冪等:べきとう）操作なら上書きしてもOK（inplace というオプションもあるくらいなので）

In [ ]:
str_concat_h0_23 = df["年"].astype(str)+"/"+df["月"].astype(str)+"/"+df["日"].astype(str)+"/"+df["時_0_23"].astype(str)
str_concat_h0_23.tail(3)

In [ ]:
pd.to_datetime(str_concat_h0_23, format="%Y/%m/%d/%H")

In [ ]:
df["date_hour"] = pd.to_datetime(str_concat_h0_23, format="%Y/%m/%d/%H")

無事にdatetime型にパースできた
## パターン1終わり

<br>
<br>
<br>

# パターン2：できるだけstringを使わずに連結し、最後だけstringに変換して日付をつくる

このくらいのデータなら正直変わらんけど、）百万行超えるくらいのデータになると、stringを使った処理がとても遅い！  だからintのままいじりたい！<BR>
groupbyなどで、keyを **string → int** にするだけで　**10倍速くこともよくある**

int で無理矢理やる

In [ ]:
df["date_int"] = df["年"]*100*100*100 + df["月"]*100*100 + df["日"]*100 + df["時"]  # stringみたいに / で区切れないので10の倍数で位をずらす

In [ ]:
pd.to_datetime(df["date_int"],format="%Y%m%d%H")  # /が無くなっている

---
公式docを見よう
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

int は想定外らしいのでキャストする  
objectにしてみる

In [ ]:
pd.to_datetime(df["date_int"].astype("O"),format="%Y%m%d%H")  # Error

In [ ]:
e=df["date_int"].astype("O")
e[0],type(e[0]) # intのままだった

object になっていたが、個別では intのままだったのでダメ  
strを試す

In [ ]:
e=df["date_int"].astype(str)
e[0],type(e[0]) # 普通にstrで良かった

In [ ]:
pd.to_datetime(df["date_int"].astype(str),format="%Y%m%d%H")  # Error

---

1-24時ではパースできません問題  

1引いて出直します

In [ ]:
df["時_0_23"] = df["時"]-1
df["時_0_23"].tail()

In [ ]:
df["date_str"] = (df["年"]*100*100*100 + df["月"]*100*100 +df["日"]*100+df["時_0_23"]).astype(str)

In [ ]:
df["date_hour"] = pd.to_datetime(df["date_str"], format="%Y%m%d%H")
df["date_hour"].head()

## パターン2終わり

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# 以上で日付(datetime) のパース完了

---


# index を datetime に変える

In [ ]:
df.head(3)

In [ ]:
df.index

今のindexは 0からの単純な整数値

indexを変えるには、 set_index もしくは df.index に直接代入すればよい

In [ ]:
df.set_index("date_hour", inplace=True)
df.index

In [ ]:
df.head(2)

邪魔な列(column)を消す

In [ ]:
df.columns

In [ ]:
df.drop(columns=["date_int","date_str","年","月","日","時","時_0_23"], inplace=True) # inplaceの方がメモリを食わないけど、
#df = df.drop(columns=["date_int","date_str","年","月","日","時","時_0_23"]) # こっちでも全然良い
df.head(3)

# やっとプロット

In [ ]:
df.plot();

<br>

## 豆腐問題 爆誕
---

japanize_matplotlibで・・・

In [ ]:
import japanize_matplotlib

In [ ]:
df.plot();

# 豆腐消えた！（たまに消えない環境も。。。）